<a href="https://colab.research.google.com/github/QueeneDelmarva/Skripsi/blob/main/Skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Skripsi Code**

## Import Libraries

## Preprocessing

## Radix Trie

### Nodes

### Radix Trie: Connecting to GSheets

### Main Fuction

In [7]:
import gspread
import string
from google.colab import auth
from google.auth import default
from oauth2client.service_account import ServiceAccountCredentials

In [8]:
def preprocess_content(content):
    content = content.translate(str.maketrans('', '', string.punctuation)).lower()
    tokens = content.split()
    tokens = [token for token in tokens if token != "noted"]
    return ''.join(tokens)

In [9]:
class EmailMetadata:
    def __init__(self, to, from_, subject, content):
        self.to = to
        self.from_ = from_
        self.subject = subject
        self.content = content

In [10]:
class RadixTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.email_metadata = []  # List to store email metadata associated with the word/phrase

    def insert(self, word, email_metadata):
        node = self
        for char in word:
            if char not in node.children:
                node.children[char] = RadixTrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.email_metadata.append(email_metadata)

    def get_email_preview(self, word):
      node = self.root
      for char in word:
        if char not in node.children:
          return None
        node = node.children[char]

        email_metadata = node.get_email_metadata()  # Retrieve the email metadata associated with the word
        if email_metadata:
          return email_metadata
        return None

    def get_email_metadata(self):
        return self.email_metadata

In [34]:
class RadixTrie:
    def __init__(self):
        self.root = RadixTrieNode()

    def load_wordlist_from_gsheet(self, gsheet_key, sheet_name="wordlist"):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('/content/skripsi-394804-a98ea9715aa4.json', scope)
        client = gspread.authorize(creds)

        worksheet = client.open_by_key(gsheet_key).worksheet('wordlist')
        email_data_list = worksheet.get_all_records()  # Fetch all rows as a list of dictionaries

        for email_data in email_data_list:
            # Preprocess the content before inserting it into the trie
            content = self._preprocess_word(email_data['Content'])

            if content:
              # Extract content into list of word
              words = content.split()

              for word in words:
                self.insert(word, email_data)

    def _preprocess_word(self, word):
    # Convert the word to lowercase
      word = word.lower()

    # Remove common greetings and noise words
      greetings = ["hi", "hello", "dear", "regards", "thanks", "thank you", "best regards"]
      if word in greetings:
        return None

    # Check for repeated information (e.g., "Re: Re: Subject")
      if word.startswith("re:"):
        return None

    # Check for null values and very short words
    # if not word or len(word) < 3:
    #     return None

      return word  # Return the processed word only


    def insert(self, word, email_metadata):
        processed_word = self._preprocess_word(word)
        if processed_word is None:
            return

        node = self.root
        for char in processed_word:
            if char not in node.children:
                node.children[char] = RadixTrieNode()
            node = node.children[char]

        node.is_end_of_word = True
        node.email_metadata = email_metadata

    #Ini cuma return none or node
    # def search(self, prefix):
    #     node = self.root
    #     current_match = ''

    #     for char in prefix:
    #         if char not in node.children:
    #             return None  # Return None instead of an empty list

    #         current_match += char
    #         node = node.children[char]
    #         print(current_match)

    #     return node  # Return the node containing the last character of the prefix

    def collect_words_from_node(self, node, prefix):
      words = []
      for char, child_node in node.children.items():
        # print("Im here fucking bitch");
        if child_node.is_end_of_word:
            # words.append((prefix + char, child_node.email_metadata))
            words.append(prefix + char)
        words.extend(self.collect_words_from_node(child_node, prefix + char))

      # print("Insert find similiar", prefix)
      # print(words)
      return words


    # Punya Airu, collect all word
    def print_all_words(self):
      all_words = self.collect_words_from_node(self.root, "")

      for word in all_words:
        print(word)

    def search(self, prefix):
      node = self.root
      current_match = ''
      matches = []

      for char in prefix:
          if char not in node.children:
              # If there is no exact match, collect all words starting with current_match
              print("the last match:", current_match)
              return self.collect_words_from_node(node, current_match)

          current_match += char
          node = node.children[char]

          if node.is_end_of_word:
              # Collect the current word if it ends at this node
              print("End of word")
              matches.append(current_match)

      print("Return karena sama sampai akhir")
      return matches

    def _get_suggested_phrases(self, node, prefix):
      phrases = []
      if node.is_end_of_word:
        phrases.append(prefix)

        for char, child_node in node.children.items():
          phrases.extend(self._get_suggested_phrases(child_node, prefix + char + " "))

        return phrases


    @staticmethod
    def levenshtein_distance(s1, s2):
        if len(s1) > len(s2):
            s1, s2 = s2, s1

        distances = range(len(s1) + 1)
        for index2, char2 in enumerate(s2):
            new_distances = [index2 + 1]
            for index1, char1 in enumerate(s1):
                if char1 == char2:
                    new_distances.append(distances[index1])
                else:
                    insert_cost = distances[index1 + 1] + 1
                    delete_cost = new_distances[-1] + 1
                    replace_cost = distances[index1] + 1
                    new_distances.append(min(insert_cost, delete_cost, replace_cost))
            distances = new_distances

        return distances[-1]

    def autocomplete(self, word):
        suggestions = self.search(word)

        print("Inside Autocomplete")
        print(suggestions)

        # Negative Cases
        if not suggestions:
            closest_word = self.get_closest_word(word)
            print(f"Suggestions: {', '.join([closest_word])}")
            user_choice = input("Is this the word you are looking for? (Y/N): ").strip().lower()
            if user_choice == "y":
                closest_word_node = self.search(closest_word)
                if closest_word_node:
                    show_preview = input("Do you want to view a snippet preview of the email data? (Y/N): ").strip().lower()
                    if show_preview == "y":
                        email_metadata_list = closest_word_node.email_metadata
                        return email_metadata_list  # Return the list of email metadata
                return [closest_word]

            while True:
                new_word = input("Enter the correct spelling or 'cancel' to cancel or 'add' to add as a new word: ").strip().lower()
                if new_word == "cancel":
                    print("Word not found. Check for misspellings.")
                    return []

                suggestions = self.search(new_word)
                if suggestions:
                    return suggestions

                user_choice = input("Word not found. Do you want to add it as a new word? (Y/N): ").strip().lower()
                if user_choice == "y":
                    email_metadata = EmailMetadata(to='', from_='', subject='', content=word)
                    self.insert(word, email_metadata)
                    gsheet_url = 'https://docs.google.com/spreadsheets/d/1RlZ8-XwwEueuCAq4nXGwq3ZmPDE-_0gP3PsroKeNFGo/edit#gid=2071884403'
                    self.save_wordlist_to_gsheet(gsheet_url, new_sheet_name="newwords")  # Save the updated wordlist
                    print("New word added to the wordlist")
                    return ["New word added to the wordlist"]

        return suggestions

    def print_email_preview(self, email_metadata):
        print("Email Preview:")
        print(f"To: {email_metadata.to}")
        print(f"From: {email_metadata.from_}")
        print(f"Subject: {email_metadata.subject}")
        print(f"Content: {email_metadata.content}")

    def get_closest_word(self, word):
        min_distance = float('inf')
        closest_word = None  # Initialize closest_word to None
        for known_word in self._get_all_words(self.root, ""):
            distance = self.levenshtein_distance(word, known_word)
            if distance < min_distance:
                min_distance = distance
                closest_word = known_word

        return closest_word  # Return None if no closest word is found

    def _get_all_words(self, node, prefix):
        words = []
        if node.is_end_of_word:
            words.append(prefix)

        for char, child_node in node.children.items():
            words.extend(self._get_all_words(child_node, prefix + char))

        return words

    def save_wordlist_to_gsheet(self, gsheet_url, new_sheet_name="newwords"):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('/content/skripsi-394804-a98ea9715aa4.json', scope)
        client = gspread.authorize(creds)

        # Open the Google Sheets document using the URL
        doc = client.open_by_url(gsheet_url)

        try:
            # Try to access the 'newwords' worksheet directly by title
            worksheet = doc.worksheet(new_sheet_name)
        except gspread.exceptions.WorksheetNotFound:
            # If the worksheet doesn't exist, create it
            worksheet = doc.add_worksheet(title=new_sheet_name, rows=1, cols=1)
            worksheet.append_row(["New Words"])  # Write the header

        # Get the new words from the trie
        new_words = list(set(self._get_all_words(self.root, "")))

        # Clear the existing content of the worksheet and write the new words
        worksheet.clear()
        worksheet.append_row(["New Words"])  # Write the header
        for word in new_words:
            worksheet.append_row([word])

    def get_email_preview(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return None
            node = node.children[char]

        return node.email_metadata

In [35]:
def main():
    trie = RadixTrie()

    # Load the wordlist from Google Sheets when starting the program
    gsheet_key = '1RlZ8-XwwEueuCAq4nXGwq3ZmPDE-_0gP3PsroKeNFGo'
    trie.load_wordlist_from_gsheet(gsheet_key)
    trie.print_all_words()

    # # while True:

    # count = 0
    # while count < 1:
    #     count += 1

    #     user_input = input("Enter a word: ").strip().lower()

    #     if user_input == "exit":
    #         break

    #     suggestions = trie.autocomplete(user_input)
        # if suggestions:
        #   print("Gua disini anjing")
        #   print(suggestions)
          # print(type(suggestions))
          # print("Gua diatas anjing")


    #       if isinstance(suggestions, EmailMetadata):  # Check if the suggestions are of type EmailMetadata
    #           for email_metadata in suggestions:
    #             trie.print_email_preview(email_metadata)
    #       else:
    #           print(f"Suggestions: {', '.join(suggestions)}")
    #     else:
    #         print("Word not found.")
    #         print("1. Correct the spelling")
    #         print("2. Add this word as a new word")
    #         choice = input("Enter your choice (1/2): ").strip()

    #         if choice == "1":
    #           while True:
    #             new_word = input("Enter the correct spelling or 'cancel' to cancel: ").strip().lower()
    #             if new_word == "cancel":
    #               break

    #             suggestions = trie.autocomplete(new_word)
    #             if suggestions and suggestions[0] != new_word:
    #               print(f"Suggestions: {', '.join(suggestions)}")
    #               break
    #             else:
    #               print("Word not found.")

    #         elif choice == "2":
    #           email_metadata = EmailMetadata(to='', from_='', subject='', content=user_input)
    #           trie.insert(user_input, email_metadata)
    #           print("New word added to the wordlist")

    # # Save new words to a different sheet in Google Sheets
    # trie.save_wordlist_to_gsheet(gsheet_key, new_sheet_name="New Words")

if __name__ == "__main__":
    main()

noted
a
and
arrange
as
airport
aviation
aerospace
confirmed
comments:
changi
centre21
crew
thanks.
transportation
time:
tumboimbella
tue
tba
please
passenger:
pickup
ground
follows:
feb
few
fa
request
road
rmark:
date:
drop-off
wed
with
01
01,
0840lt
03
2023
qg-552
quwense
eta
location:
limousine
luggage
seletar
singapore
sbac
business
1797405,
1930lt
vehicles:
jhasletha
jan
mpv
has
her.
hotel
31
31jan
/
-
